# Tutorial 01: Hello Mean Field Games

This is the simplest possible MFG example using the Linear-Quadratic (LQ) framework.

## Learning Objectives

By the end of this tutorial, you will understand:
- How to define an MFG problem using `ExampleMFGProblem`
- How to solve it using the `solve_mfg()` function
- How to inspect and validate the solution
- How to check mass conservation

## Mathematical Problem

We solve a Linear-Quadratic Mean Field Game on the domain $[0,1]$ with:
- **Hamiltonian**: $H(p) = \frac{1}{2}|p|^2$
- **Coupling**: $\lambda \cdot m$ (congestion cost)
- **Initial density**: Gaussian centered at $x=0.5$
- **Terminal cost**: $g(x) = \frac{1}{2}(x - 0.5)^2$

**Time estimate**: 10 minutes

## Step 1: Import Dependencies

In [ ]:
import numpy as np

from mfg_pde import ExampleMFGProblem, solve_mfg

## Step 2: Create the Problem

The `ExampleMFGProblem` is a pre-configured Linear-Quadratic MFG problem. It's the simplest way to get started with MFG_PDE.

In [ ]:
problem = ExampleMFGProblem(
    # Spatial domain: [0, 1]
    xmin=0.0,
    xmax=1.0,
    Nx=50,  # Number of spatial grid points
    # Time horizon: [0, 1]
    T=1.0,
    Nt=50,  # Number of time steps
    # Diffusion coefficient (controls agent randomness)
    sigma=0.1,
    # Congestion parameter (controls interaction strength)
    lam=0.5,
)

print("Problem created:")
print(f"  Domain: [{problem.xmin}, {problem.xmax}]")
print(f"  Grid: {problem.Nx} spatial points, {problem.Nt} time steps")
print(f"  Diffusion σ = {problem.sigma}")
print(f"  Congestion λ = {problem.lam}")

## Step 3: Solve the MFG

The `solve_mfg()` function solves the coupled HJB-FP system using Picard fixed-point iteration:
1. Solve HJB equation backward in time given density $m$
2. Solve Fokker-Planck equation forward in time given value $u$
3. Iterate until convergence

In [ ]:
print("Solving MFG system...\n")
result = solve_mfg(problem, verbose=True)
print("\nSolution completed!")

## Step 4: Inspect the Solution

The result object contains:
- `U`: Value function (solution to HJB equation)
- `M`: Density (solution to Fokker-Planck equation)
- Convergence information

In [ ]:
print("=" * 70)
print("SOLUTION SUMMARY")
print("=" * 70)
print()
print(f"Converged: {result.converged}")
print(f"Iterations: {result.iterations}")
print(f"Final residual: {result.residual:.6e}")
print()
print("Solution arrays:")
print(f"  U (value function): {result.U.shape}")
print(f"  M (density): {result.M.shape}")

## Step 5: Validate Mass Conservation

A key sanity check for MFG solutions is mass conservation. The total mass should remain constant at 1.0 throughout time:

$$\int_0^1 m(t,x) dx = 1 \quad \forall t \in [0,T]$$

In [ ]:
# Mass conservation check
total_mass = np.sum(result.M[-1, :]) * problem.dx
print(f"\nFinal mass (should be ≈1.0): {total_mass:.6f}")

if abs(total_mass - 1.0) < 0.01:
    print("✓ Mass conservation satisfied!")
else:
    print("⚠ Mass conservation violated - check numerical settings")

## Step 6: Visualize the Solution

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot value function at final time
axes[0].plot(problem.xSpace, result.U[-1, :])
axes[0].set_xlabel("x")
axes[0].set_ylabel("u(T, x)")
axes[0].set_title("Terminal Value Function")
axes[0].grid(True, alpha=0.3)

# Plot density evolution
X, T = np.meshgrid(problem.xSpace, problem.tSpace)
contour = axes[1].contourf(X, T, result.M, levels=20, cmap="viridis")
axes[1].set_xlabel("x")
axes[1].set_ylabel("t")
axes[1].set_title("Density Evolution m(t,x)")
plt.colorbar(contour, ax=axes[1])

plt.tight_layout()
plt.show()

## Summary

### What You Learned

1. How to create an MFG problem using `ExampleMFGProblem`
2. How to solve it with `solve_mfg()`
3. How to inspect the solution (U, M, convergence)
4. How to check mass conservation
5. How to visualize the results

### Key Takeaways

- MFG solutions consist of two coupled functions: value $u(t,x)$ and density $m(t,x)$
- Picard iteration alternates between HJB and FP solves until convergence
- Mass conservation is a critical validation metric
- The solution shows how agents' optimal behavior (encoded in $u$) affects population distribution (encoded in $m$)

### Next Steps

Proceed to **Tutorial 02: Custom Hamiltonian** to learn how to define your own MFG problems.